In [ ]:
import yaml
import numpy as np
import pandas as pd
import math

In [2]:
filepaths_HOME = {'constraints': '/Users/jacobejsing/saam_opt/cvx_constraints.yaml', \
                 'names': '/Users/jacobejsing/saam_opt/strat_names.yaml', \
                 'scenarios': '/Users/jacobejsing/saam_opt/scenarios.csv'}

filepaths_WORK = {'constraints': 'C:/Users/jwe/saam/cvx_constraints.yaml', \
                 'names': 'C:/Users/jwe/saam/strat_names.yaml', \
                 'scenarios': 'C:/Users/jwe/saam/SAAM_data/scenarios.csv'}

filepaths = filepaths_HOME

### Read SAAM optimisation constraints from yaml

In [3]:
with open(filepaths['constraints'], 'r') as f:
    d = yaml.safe_load(f)

arrays = {}
for key in d:
    arrays[key] = np.array(d[key])

In [4]:
# Replace inf for UB
arrays['UB'][arrays['UB'] == np.inf] = 1e9

In [5]:
# Move FX reserve size to equality constraints

arrays['A_eq'] = np.concatenate([arrays['A_ineq'][0:1,:], arrays['A_eq']])
arrays['b_eq'] = np.concatenate([arrays['b_ineq'][0:1,:], arrays['b_eq']])

arrays['A_ineq'] = arrays['A_ineq'][1:, :]
arrays['b_ineq'] = arrays['b_ineq'][1:, :]

In [6]:
display(arrays['A_eq'].shape)
display(arrays['A_ineq'].shape)

(10, 85)

(8, 85)

### Read strategy names from yaml

In [7]:
with open(filepaths['names'], 'r') as f:
    L = yaml.safe_load(f)

names = [elem[0] for elem in L]

### Read SAAM scenarios from csv

In [8]:
df = pd.read_csv(filepaths['scenarios'], header=None)
df.columns = names
scenarios = df.values[:10000, :]
del df

### Run SAAM optimisation with CVX

In [9]:
from cvxopt import matrix, solvers

In [10]:
P = 2 * matrix([ [2, .5], [.5, 1] ])
q = matrix([1.0, 1.0])
G = matrix([[-1.0,0.0],[0.0,-1.0]])
h = matrix([0.0,0.0])
A = matrix([1.0, 1.0], (1,2))
b = matrix(1.0)
sol = solvers.qp(P, q, G, h, A, b)
print(sol['x'])

     pcost       dcost       gap    pres   dres
 0:  1.8889e+00  7.7778e-01  1e+00  3e-16  2e+00
 1:  1.8769e+00  1.8320e+00  4e-02  2e-16  6e-02
 2:  1.8750e+00  1.8739e+00  1e-03  2e-16  5e-04
 3:  1.8750e+00  1.8750e+00  1e-05  6e-17  5e-06
 4:  1.8750e+00  1.8750e+00  1e-07  2e-16  5e-08
Optimal solution found.
[ 2.50e-01]
[ 7.50e-01]



### Inspect individual constraints

In [11]:
j = 0

selection = []
for ix, name in enumerate(names):
    if arrays['A_eq'][j, ix]:
        #print(name)
        selection.append(name)
        
#display(arrays['A_eq'][j,])
display(arrays['b_eq'][j,])

array([442544.22022029])

In [12]:
info = pd.DataFrame(arrays['A_eq'][j,], index = names, columns=['coef'])
info.sort_values('coef').head(10)

,coef
EURDKK,0.0
INDSKUD,0.0
FOLIO,0.0
SPX_FUT,0.0
EQUITY_FUT,0.0
DKKSWPY5,0.0
CARRY_USDEURM3,0.0
DKKSWPW1,0.0
DKKSWPY2,0.0
DKKSWPY10,0.0


In [13]:
type(scenarios)

numpy.ndarray

In [14]:
N = scenarios.shape[1]

P = matrix(2 * np.cov(scenarios.T))
q = matrix(np.zeros((N, 1)))

#G = matrix(arrays['A_ineq'])
#h = matrix(arrays['b_ineq'])

# Including upper and lower bounds in inequality constraints
#G = matrix(np.zeros((1, 85)))
#h = matrix(0.0)

#G = matrix(arrays['A_ineq'][1:,])
#h = matrix(arrays['b_ineq'][1:,])

r_hat = np.mean(scenarios, axis=0);
min_return = -1200.0 * np.ones((1,1));

G = matrix(np.concatenate([arrays['A_ineq'], np.identity(N), -np.identity(N), -r_hat.reshape(1, N)]))
h = matrix(np.concatenate([arrays['b_ineq'], arrays['UB'], -arrays['LB'], -min_return]))

A = matrix(arrays['A_eq'][1:,:])
b = matrix(arrays['b_eq'][1:,:])

solvers.options['show_progress'] = True
sol = solvers.qp(P, q, G, h, A, b)
sol['status']

     pcost       dcost       gap    pres   dres
 0:  1.3723e+16 -6.8084e+18  6e+19  9e-01  6e+09
 1:  7.6170e+14 -1.2581e+18  1e+19  1e-01  1e+09
 2:  2.9251e+11 -3.1845e+16  2e+17  3e-03  2e+07
 3:  3.1542e+08 -1.0756e+15  6e+15  8e-05  6e+05
 4:  4.1446e+07 -2.6569e+14  5e+14  3e-06  2e+04
 5:  4.4500e+07 -3.0959e+13  5e+13  3e-07  2e+03
 6:  4.2201e+07 -1.5914e+13  3e+13  2e-07  1e+03
 7:  1.5986e+07 -1.2456e+13  2e+13  1e-07  8e+02
 8:  1.2225e+07 -1.9845e+12  3e+12  5e-09  4e+01
 9:  1.2027e+07 -2.3515e+10  3e+10  5e-11  4e-01
10:  1.2020e+07 -2.6799e+08  3e+08  7e-13  5e-03
11:  1.1669e+07 -2.4364e+07  4e+07  8e-14  6e-04
12:  1.0206e+07  2.9109e+06  9e+06  1e-14  1e-04
13:  8.5862e+06  7.3272e+06  1e+06  1e-15  1e-05
14:  8.1394e+06  7.9243e+06  2e+05  2e-16  8e-06
15:  8.0359e+06  8.0132e+06  3e+04  2e-16  1e-05
16:  8.0245e+06  8.0238e+06  7e+02  2e-16  1e-05
17:  8.0242e+06  8.0242e+06  7e+01  2e-16  9e-06
18:  8.0242e+06  8.0242e+06  4e+01  2e-16  1e-05
19:  8.0242e+06  8.02

'unknown'

In [15]:
%%timeit
solvers.options['show_progress'] = False
sol = solvers.qp(P, q, G, h, A, b)

12.3 ms ± 20.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
stdev = math.sqrt(sol['primal objective'])
pnl = np.dot(r_hat, sol['x'])
f"Pnl {pnl[0]}, Stdev {stdev}"

'Pnl -1200.0000000000005, Stdev 2832.701550342746'

### Functions

In [23]:
def minimize_risk(r_hat, min_return):
    
    min_return = min_return * np.ones((1,1))
    
    G = matrix(np.concatenate([arrays['A_ineq'], np.identity(N), -np.identity(N), -r_hat.reshape(1, N)]))
    h = matrix(np.concatenate([arrays['b_ineq'], arrays['UB'], -arrays['LB'], -min_return]))
    
    A = matrix(arrays['A_eq'][1:,:])
    b = matrix(arrays['b_eq'][1:,:])

    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)
    return solution

def extract_results(solution, r_hat):
    results = {}
    results['w'] = solution['x']
    results['stdev'] = math.sqrt(solution['primal objective'])
    results['return'] = np.dot(r_hat, results['w'])[0]
    results['VaR'] = np.percentile(np.dot(scenarios, results['w']), 5, axis=0)[0]
    return results

In [24]:
r_hat = np.mean(scenarios, axis=0)

frontier_data = []

for min_return in list(range(-1200, -300, 10)):
    solution = minimize_risk(r_hat, min_return)
    results = extract_results(solution, r_hat)
    frontier_data.append([min_return, results['return'], results['stdev'], results['VaR']])
    #print(min_return, results['return'], results['stdev'])

In [26]:
frontier = pd.DataFrame(frontier_data, columns=['min_return', 'return', 'stdev', 'VaR'])
frontier.head()

,min_return,return,stdev,VaR
0,-1200,-1200.0,2832.701550,-5378.947696
1,-1190,-1190.0,2833.278636,-5370.336780
2,-1180,-1180.0,2834.333761,-5370.597142
3,-1170,-1170.0,2836.123213,-5352.806440
4,-1160,-1160.0,2838.499173,-5352.190851


In [28]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [29]:
frontier.plot(x="VaR", y="return", kind="scatter", grid=True);